In [3]:
import requests
import re
import string

import nltk
from nltk.corpus import stopwords
stop_words=stopwords.words('english')
stop_words.extend(["ad","quot","ca"])
from nltk.tokenize import word_tokenize
nltk.download('words')
words = set(nltk.corpus.words.words())

import matplotlib.pyplot as plt

import pandas as pd
import csv

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\Tom\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


Get speeches from website

In [2]:
# To open the csv file with the cleaned speeches use 
# with open('speeches_clean.csv', newline = '') as file:
#     reader = csv.reader(file)
#     "name" = list(reader)


with open('C:\\Users\\Tom\\Documents\\MEGAsync\\KUL\\M2S2\\MDA\\Project\\Speeches.csv', newline='') as file:
    reader = csv.reader(file)
    data = list(reader)
data = data[62:len(data)-1] # Start with first inaugural address and last entry is not a speech

In [3]:
# Remove unnecessary items from the list, result = ['title', 'url', 'date'] for each speech
for i in reversed(range(len(data))):
    data[i] = [x for x in data[i] if not x == '' if not 'mp3' in x if not 'pdf' in x if not 'PDF' in x]
    if len(data[i]) != 3:
        data.remove(data[i])

In [4]:
# Get the speeches from the website (this step takes 3-5 min)

headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; rv:25.0) Gecko/20100101 Firefox/25.0'}

speeches_raw = list()
for i in range(len(data)):
    r = requests.get(data[i][1], headers = headers)
    speeches_raw.append((data[i][0],data[i][2],r.text))
    

Function for cleaning the speeches

In [5]:
def cleaning_speeches(speeches):
    # Remove html code and white spaces
    speeches = speeches.lower()
    speeches = re.sub('<.*>', '', speeches)
    speeches = re.sub('\((.*?\))', '', speeches)
    speeches = re.sub('\[.*?\]', '', speeches)
    speeches = re.sub('[%s]' % re.escape(string.punctuation), ' ', speeches)
    speeches = re.sub('\w*\d\w*', '', speeches)
    speeches = re.sub('\n*\r*\t*', '', speeches)
    # Remove stop words
    tokenized_speeches=word_tokenize(speeches)
    speeches_wo_stopwords= [w for w in tokenized_speeches if not w in stop_words] 
    speeches_wo_stopwords=' '.join(speeches_wo_stopwords) 
    # Remove non-English words
    speeches_wo_nonwords = [w for w in nltk.wordpunct_tokenize(speeches_wo_stopwords) if w.lower() in words or not w.isalpha()]
    speeches_wo_nonwords = ' '.join(speeches_wo_nonwords)
    return speeches_wo_nonwords

Cleaning the speeches

In [6]:
speeches_clean = list()
for i in range(0,len(speeches_raw)):
    clean_txt = cleaning_speeches(speeches_raw[i][2])
    speeches_clean.append((speeches_raw[i][0],speeches_raw[i][1],clean_txt[180:-180])) #remove the first and last few words, since they are not part of the speeches (unfortunately, not the same amount for each speech though)

In [7]:
speeches_clean[0][2]

'oath office solemnly swear execute office president united faithfully best ability preserve protect defend constitution united help god stand today task us grateful trust mindful borne thank president bush service nation well generosity shown throughout transition forty four taken presidential oath spoken rising prosperity still peace yet every often oath taken amidst gathering raging carried simply skill vision high office people faithful true founding must generation midst crisis well understood nation war far reaching network violence hatred economy badly consequence greed irresponsibility part also collective failure make hard prepare nation new age lost shed health care costly fail many day evidence ways use energy strengthen threaten planet crisis subject data statistics less measurable less profound sapping confidence across land nagging fear decline inevitable next generation must lower today say face real serious many met easily short span time know met day gather chosen hope

In [8]:
df = pd.DataFrame(speeches_clean)
df.columns = ['Date', 'Title', 'Text']

In [4]:
#To open the csv file with the cleaned speeches use 
with open('speeches_clean.csv', newline = '') as file:
    reader = csv.reader(file)
    speech_test = list(reader)